In [1]:
!pip install numpy==1.18.5 --quiet

In [2]:
!pip install pandas --upgrade --quiet

In [3]:
!pip install matplotlib seaborn --upgrade --quiet

In [4]:
!pip install tensorflow --upgrade --quiet

In [15]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import tensorflow as tf
from sklearn.model_selection import cross_val_score, train_test_split

In [16]:
def load_data(nao_index_train_path = './nao_index_train.npy',
              tas_train_path = './tas_train.npy',
              psl_train_path = './psl_train.npy'):

    x1 = np.load(tas_train_path)
    x2 = np.load(psl_train_path)
    y = np.load(nao_index_train_path)
    return x1, x2, y

temp_train, pres_train, nao_index_train = load_data()

In [17]:
np.shape(temp_train), np.shape(pres_train), np.shape(nao_index_train)

((900, 2322), (900, 2322), (900,))

In [18]:
# PCA with 50 components
pca = PCA(n_components=50)
pca.fit(temp_train)
temp_train = pca.transform(temp_train)
pca.fit(pres_train)
pres_train = pca.transform(pres_train)

In [19]:
## Reshape the data as per lat & lon
#temp_train_reshaped = np.reshape(temp_train,(900,43,54))
#pres_train_reshaped = np.reshape(pres_train,(900,43,54))

In [20]:
X_comb = np.concatenate((temp_train,pres_train), axis =1)
y = nao_index_train
np.shape(X_comb)

(900, 100)

In [21]:
X_train, X_val, y_train, y_val= train_test_split(X_comb, y, test_size=0.1, random_state=42)

In [22]:
np.shape(X_train), np.shape(X_val), np.shape(y_train), np.shape(y_val)

((810, 100), (90, 100), (810,), (90,))

In [25]:
X_train_tensor = tf.convert_to_tensor(X_train, dtype=np.float32, dtype_hint=None, name=None)
y_train_tensor = tf.convert_to_tensor(y_train, dtype=np.float32, dtype_hint=None, name=None)

# def tensor_convert(x1 = temp_train, x2 = pres_train):
#     y1 = tf.convert_to_tensor(x1, dtype=np.float32, dtype_hint=None, name=None)
#     y2 = tf.convert_to_tensor(x2, dtype=np.float32, dtype_hint=None, name=None)
#     return y1, y2

# temp_tensor, pres_tensor = tensor_convert()

In [26]:
train_ds = tf.data.Dataset.from_tensor_slices(X_train_tensor, y_train_tensor)
train_ds

TypeError: from_tensor_slices() takes 1 positional argument but 2 were given